In [5]:
import random
import numpy as np

In [2]:
total_col_count = 1245
total_row_count = 100

fat_dof = 5
lean_A_dof = 6
lean_B_dof = 7
lean_C_dof = 8

fat_needs_dicretization = True
lean_A_needs_dicretization = False
lean_B_needs_dicretization = False
lean_C_needs_dicretization = False

reg_fat_count = 222
reg_lean_A_count = 13
reg_lean_B_count = 12
reg_lean_C_count = 11

heur_fat_count = 542
heur_lean_A_count = 42
heur_lean_B_count = 34
heur_lean_C_count = 0

contr_fat_count = 200
contr_lean_A_count = 124
contr_lean_B_count = 0
contr_lean_C_count = 45

In [3]:
assert reg_fat_count + heur_fat_count + contr_fat_count +\
reg_lean_A_count + heur_lean_A_count + contr_lean_A_count +\
reg_lean_B_count + heur_lean_B_count + contr_lean_B_count +\
reg_lean_C_count + heur_lean_C_count + contr_lean_C_count \
== total_col_count

In [7]:
column_range_counts = \
[reg_fat_count,
 reg_lean_A_count,
 reg_lean_B_count,
 reg_lean_C_count,
 heur_fat_count,
 heur_lean_A_count,
 heur_lean_B_count,
 heur_lean_C_count,
 contr_fat_count,
 contr_lean_A_count,
 contr_lean_B_count,
 contr_lean_C_count
]

range_needs_discretizations = \
[fat_needs_dicretization,
 lean_A_needs_dicretization,
 lean_B_needs_dicretization,
 lean_C_needs_dicretization
] * 3

range_dofs = \
[fat_dof,
 lean_A_dof,
 lean_B_dof,
 lean_C_dof    
] * 3

range_bounds = np.cumsum(column_range_counts).tolist()
range_bounds

[222, 235, 247, 258, 800, 842, 876, 876, 1076, 1200, 1200, 1245]

In [9]:
# So the subsequent contingues ranges of columns
# in the hypothetical data can be characterized by:
# - lower (left) and upper (right) column indeces which form
# the boundary (left-closed, right-open) of each range,
# - number of dof of columns in that range,
# - and whether or not columns in that range need to be discretized.
# If the answer to the latter question is positive, we will assume that
# the range will also need to be further split into smaller subranges
# that correspond to the notion of tiles in downstream computation.

for range_bound_1, range_bound_2, range_dof, range_needs_dicretization\
in zip([0] + range_bounds[:-1], range_bounds, range_dofs, range_needs_discretizations):
    print(range_bound_1, range_bound_2, range_dof, range_needs_dicretization)

0 222 5 True
222 235 6 False
235 247 7 False
247 258 8 False
258 800 5 True
800 842 6 False
842 876 7 False
876 876 8 False
876 1076 5 True
1076 1200 6 False
1200 1200 7 False
1200 1245 8 False


In [12]:
fat_tile_width = 100

tile_bounds = [0]
tile_group_bounds = [0]
tile_needs_discretizations = []
tile_dofs = []
in_fat_tile_group = True

for range_bound_1, range_bound_2, range_dof, range_needs_dicretization\
in zip([0] + range_bounds[:-1], range_bounds, range_dofs, range_needs_discretizations):
    
    if range_needs_dicretization:
        if not in_fat_tile_group:
            tile_group_bounds.append(len(tile_bounds) - 1)
        in_fat_tile_group = True
        number_of_tiles = (range_bound_2 - range_bound_1) // fat_tile_width
        for _ in range(number_of_tiles):
            tile_bounds.append(tile_bounds[-1] + fat_tile_width)
            tile_needs_discretizations.append(True)
            tile_dofs.append(range_dof)
        tile_group_bounds.append(len(tile_bounds) - 1)
        width_of_leftover_margin = (range_bound_2 - range_bound_1) % fat_tile_width
        if width_of_leftover_margin:
            tile_bounds.append(tile_bounds[-1] + width_of_leftover_margin)
            tile_needs_discretizations.append(True)
            tile_dofs.append(range_dof)
    else:
        in_fat_tile_group = False
        tile_bounds.append(tile_bounds[-1] + (range_bound_2 - range_bound_1))
        tile_needs_discretizations.append(False)
        tile_dofs.append(range_dof)

if not in_fat_tile_group:
    tile_group_bounds.append(len(tile_bounds) - 1)

In [13]:
len(tile_bounds)

21

In [14]:
tile_bounds

[0,
 100,
 200,
 222,
 235,
 247,
 258,
 358,
 458,
 558,
 658,
 758,
 800,
 842,
 876,
 876,
 976,
 1076,
 1200,
 1200,
 1245]

In [15]:
tile_group_bounds

[0, 2, 6, 11, 15, 17, 20]

In [17]:
len(tile_needs_discretizations)

20

In [18]:
tile_needs_discretizations

[True,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False]

In [19]:
len(tile_dofs)

20

In [20]:
tile_dofs

[5, 5, 5, 6, 7, 8, 5, 5, 5, 5, 5, 5, 6, 7, 8, 5, 5, 6, 7, 8]

In [15]:
arr = np.zeros((total_row_count, total_col_count))

for tile_bound_1, tile_bound_2, discretization, dof\
in zip(tile_bounds[:-1],tile_bounds[1:], tile_discretizations, tile_dofs):
    print(tile_bound_1, tile_bound_2, discretization, dof)
    for column_index in range(tile_bound_1, tile_bound_2):
        if discretization:
            for row_index in range(total_row_count):
                arr[row_index, column_index] = random.gauss(0, 1)
        else:
            for row_index in range(total_row_count):
                arr[row_index, column_index] = random.randint(0, dof-1)

0 100 True 5
100 200 True 5
200 222 True 5
222 235 False 6
235 247 False 7
247 258 False 8
258 358 True 5
358 458 True 5
458 558 True 5
558 658 True 5
658 758 True 5
758 800 True 5
800 842 False 6
842 876 False 7
876 876 False 8
876 976 True 5
976 1076 True 5
1076 1200 False 6
1200 1200 False 7
1200 1245 False 8


In [19]:
arr[:,222]

array([5., 1., 3., 5., 2., 1., 3., 1., 2., 5., 3., 3., 5., 5., 1., 0., 1.,
       0., 3., 0., 2., 5., 3., 3., 4., 1., 3., 1., 5., 2., 2., 3., 1., 0.,
       0., 1., 0., 0., 5., 4., 0., 5., 4., 2., 2., 0., 0., 3., 0., 0., 1.,
       1., 0., 3., 1., 3., 0., 2., 4., 5., 0., 4., 4., 5., 1., 0., 4., 3.,
       4., 1., 1., 0., 1., 1., 4., 2., 1., 0., 2., 5., 0., 4., 4., 5., 1.,
       2., 0., 3., 2., 4., 1., 3., 2., 5., 2., 2., 3., 4., 2., 0.])

In [16]:
for column in arr.T:
    print()

In [64]:
for tile_bound_1, tile_bound_2, discretization, dof\
in zip(tile_bounds[:-1],tile_bounds[1:], tile_discretizations, tile_dofs):
    for column_index in range(tile_bound_1, tile_bound_2):
        if not discretization:
            assert set(arr[:, column_index]) == set(range(1, dof+1))

In [68]:
np.savetxt("dummy_.csv", arr, delimiter=",")

In [20]:
import pandas as pd
df = pd.DataFrame(arr)
for tile_bound_1, tile_bound_2, discretization, dof\
in zip(tile_bounds[:-1],tile_bounds[1:], tile_discretizations, tile_dofs):
    for column_index in range(tile_bound_1, tile_bound_2):
        if not discretization:
            df[column_index] = df[column_index].astype('int')

In [21]:
df.to_csv("dummy.csv", header=False, index=False)